<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langgraph langchain_openai openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00


In [ ]:
import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
import json

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

In [ ]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str
    results: Dict[str, Any]


def create_agent(name: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are {name}, an AI agent responsible for processing user input. "
                  "Analyze the input and provide your assessment."),
        ("human", "{input}")
    ])

    def agent_function(state: AgentState):
        current_input = state["current_input"]

        messages = prompt.format_messages(
            name=name,
            input=current_input
        )

        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"][name] = response.content

        return new_state

    return agent_function

first_agent = create_agent("DataAnalyzer")

workflow = StateGraph(AgentState)

workflow.add_node("analyze", first_agent)

workflow.set_entry_point("analyze")

workflow.add_edge("analyze", END)

app = workflow.compile()

In [ ]:
initial_state = {
    "messages": [],
    "current_input": "This is a test input that needs to be analyzed.",
    "results": {}
}

result = app.invoke(initial_state)

print("Results:", result["results"])

Results: {'DataAnalyzer': 'Thank you for providing the input. As an AI DataAnalyzer, I can see that the input is a simple text string indicating that it is a test input that needs to be analyzed. It does not contain any specific data or information for analysis. If you have any specific data or questions you would like me to analyze, please provide them for further assessment.'}
